In [ ]:
import re

import rtsvg
rt    = rtsvg.RACETrack()

paths = ['M 10 10 L 200 200',
         'M 20 30 L 100 140 L 200 235',
         'M 15 50 C 100 140 150 250 240 230',
         'M 20 70 40 90 100 180 L 150 200 200 300',
         'M 20 120 40 150 100 200 C150,240 200 340 250 350 L 270 380',
         'M30,180l130,  170L300,420']

# These are just the raw paths
_w_, _h_ = 320, 512
_svg_    = [f'<svg x="0" y="0" width="{_w_}" height="{_h_}"><rect x="0" y="0" width="{_w_}" height="{_h_}" fill="white"/>']
for _path_ in paths: _svg_.append(f'<path d="{_path_}" stroke="black" stroke-width="1.0" fill="none" />')
_svg_.append('</svg>')
_svg0_ = ''.join(_svg_)

# This is how we'd like it to work ... but SVG won't smoothly animate unless each element in the path is the same type (and the same number)
_svg_    = [f'<svg x="0" y="0" width="{_w_}" height="{_h_}"><rect x="0" y="0" width="{_w_}" height="{_h_}" fill="white"/>']
_svg_.append(f'<path stroke="black" stroke-width="1.0" fill="none">')
_svg_.append(f'<animate attributeName="d" values="{';'.join(paths)}" dur="4s" repeatCount="indefinite"/>')
_svg_.append(f'</path>')
_svg_.append('</svg>')
_svg1_ = ''.join(_svg_)

#rt.tile([_svg0_, _svg1_], spacer=10)

In [ ]:
#
# svgPathToComponents_limited() - parse an SVG path description into its primitive components
# - limited -- only works on a subset of SVG path commands
# - only accepts a single "M"ove command at the start
# - returns the results as a list of tuples encapsulating each component
#
def svgPathToComponents_limited(_path_):
    _components_ = []
    def is_float(s):
        try:               float(s)            
        except ValueError: return False
        return True
    def are_floats(l): return all(is_float(x) for x in l)
    _parts_ = re.split(r'([MmLlCc ,])',_path_)
    _parts_ = [x for x in _parts_ if x != '' and x != ',' and x !=' ']
    if _parts_[0] == 'M' or _parts_[0] == 'm': _parts_.pop(0)
    else: raise ValueError(f'Path "{_path_}" should start with an M or m')
    x = float(_parts_.pop(0))
    y = float(_parts_.pop(0))
    while len(_parts_) > 0:
        # Line Component
        if   _parts_[0] == 'L' or _parts_[0] == 'l' and len(_parts_) >= 3 and are_floats(_parts_[1:3]):
            _cmd_, x1, y1 = _parts_[0], float(_parts_[1]), float(_parts_[2])
            _parts_ = _parts_[3:]
            if _cmd_ == 'l': x1, y1 = x+x1, y+y1 # lower case is relative
            _components_.append(('line', x, y, x1, y1))
        # Bezier Component
        elif _parts_[0] == 'C' or _parts_[0] == 'c' and len(_parts_) >= 7 and are_floats(_parts_[1:7]):
            _cmd_, xc0, yc0, xc1, yc1, x1, y1 = _parts_[0], float(_parts_[1]), float(_parts_[2]), float(_parts_[3]), float(_parts_[4]), float(_parts_[5]), float(_parts_[6])
            _parts_ = _parts_[7:]
            if _cmd_ == 'c': xc0, yc0, xc1, yc1, x1, y1 = x+xc0, y+yc0, x+xc1, y+yc1, x+x1, y+y1 # lower case is relative
            _components_.append(('bezier', x, y, xc0, yc0, xc1, yc1, x1, y1))
        # Line Component (w/out the command)
        elif len(_parts_) >= 2 and are_floats(_parts_[:2]):
            x1, y1 = float(_parts_[0]), float(_parts_[1])
            _parts_ = _parts_[2:]
            _components_.append(('line', x, y, x1, y1))
        # Exception
        else: raise ValueError(f'Path "{_path_}" contains an unexpected command "{_parts_[0]}"')
        x, y = x1, y1
    # Return the components
    return _components_

for _path_ in paths: svgPathToComponents_limited(_path_)